# Capstone Project

### Business Problem: 

 A friend of mine is interested in opening a new restaurant within a 2 mile radius of New York, NY. He tasked me with finding the best location to open the restaurant. 

### Target Audience:
This program will be broad enough to assist others in answering the question, “if I wanted to open a restaurant within a X mile radius of city-Y, and state-Z, where would be the best location(s)?” Companies such as Waffle House, Golden Corral, McDonalds would find this program beneficial.

### Data:
A friend of mine is interested in opening a new restaurant within a 2 mile radius of New York, NY. He tasked me with finding the best location to open the restaurant. This program will be broad enough to assist others in answering the question, “if I wanted to open a restaurant within a X mile radius of city-Y, and state-Z, where would be the best location(s)?” Companies such as Waffle House, Golden Corral, McDonalds would find this program beneficial.

### User updates to customize program to their choosing.

In [245]:
address = 'Charlotte, NC'
radius = 2
NumberOfClusters = 2 #Not Including Restaurants

### Read in necessary modules and connect to API

In [246]:
from requests import get
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
from IPython.display import HTML
import requests
#Four Square Info
CLIENT_ID = 'UDIPNIDGY1FHFKHUGNSFX3D2AZXRW0FOKG3DYDMOJALXNZHO' # your Foursquare ID
CLIENT_SECRET = 'IG4CPTMWECQXIHKOJCW1QSQK25BBGZ1XC2VT4OYUUNUR50QP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version



### Get Latitude and Longitude Locations of requested address

In [247]:
geolocator = Nominatim(user_agent="restaurant_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Latitude/Longitude: " + str(latitude) + "/" + str(longitude))

Latitude/Longitude: 35.2270869/-80.8431268


### Convert miles (specified by user) to meters

In [248]:
miles = int(round(radius*1609.34))

### Create/Execute Function to make a call to the API and gather all venue information

In [257]:
def getNearbyVenues(names, latitudes, longitudes, radius=miles):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
       
        # return only relevant information for each nearby venue
        itemList = [(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results]
                    
        venues_list.append(itemList)
                   

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

venues = getNearbyVenues(names=[address],
                                   latitudes=[latitude],
                                   longitudes=[longitude]
                                  )
venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Charlotte, NC",35.227087,-80.843127,Blumenthal Performing Arts Center,35.227930,-80.841951,Performing Arts Venue
1,"Charlotte, NC",35.227087,-80.843127,Belk Theater,35.227711,-80.841663,Concert Hall
2,"Charlotte, NC",35.227087,-80.843127,The Capital Grille,35.228216,-80.841974,American Restaurant
3,"Charlotte, NC",35.227087,-80.843127,The Punch Room,35.226354,-80.841747,Hotel Bar
4,"Charlotte, NC",35.227087,-80.843127,Vapiano,35.226234,-80.844124,Italian Restaurant


### Standardize Venues to convert venue categories to restaurants where necessary

In [258]:
#venueRestaurant = venues.loc[venues['Venue Category'].str.find("Restaurant") != -1]
#venueNonRestaurant = venues.loc[venues['Venue Category'].str.find("Restaurant") == -1]
#venueRestaurant.head(25)
TurnToRestaurant = ["Pizza","Sandwich","Restaurant", "Deli", "Burger", "Noodle"]
NewCategories = []
for i in venues['Venue Category'].tolist():
    FindIt = [j for j in TurnToRestaurant if (i.upper()).find(j.upper()) != -1]
    if FindIt:
        NewCategories.append("Restaurant")
    else:
        NewCategories.append(i)
        
        
            
    

venues['Venue Category'] = NewCategories
venueNonRestaurant = venues.copy()
venueNonRestaurant.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Charlotte, NC",35.227087,-80.843127,Blumenthal Performing Arts Center,35.227930,-80.841951,Performing Arts Venue
1,"Charlotte, NC",35.227087,-80.843127,Belk Theater,35.227711,-80.841663,Concert Hall
2,"Charlotte, NC",35.227087,-80.843127,The Capital Grille,35.228216,-80.841974,Restaurant
3,"Charlotte, NC",35.227087,-80.843127,The Punch Room,35.226354,-80.841747,Hotel Bar
4,"Charlotte, NC",35.227087,-80.843127,Vapiano,35.226234,-80.844124,Restaurant


### Cluster the venues by K-means

In [251]:
venueNonRestaurant.groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Bakery,1,1,1,1,1,1
Basketball Stadium,1,1,1,1,1,1
Café,1,1,1,1,1,1
Coffee Shop,1,1,1,1,1,1
Concert Hall,1,1,1,1,1,1
Creperie,1,1,1,1,1,1
Farmers Market,1,1,1,1,1,1
Gym,1,1,1,1,1,1
Gym / Fitness Center,1,1,1,1,1,1


In [252]:
print('There are {} uniques categories.'.format(len(venueNonRestaurant['Venue Category'].unique())))

There are 18 uniques categories.


In [253]:
venueNonRestaurant.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Charlotte, NC",35.227087,-80.843127,Blumenthal Performing Arts Center,35.227930,-80.841951,Performing Arts Venue
1,"Charlotte, NC",35.227087,-80.843127,Belk Theater,35.227711,-80.841663,Concert Hall
2,"Charlotte, NC",35.227087,-80.843127,The Capital Grille,35.228216,-80.841974,Restaurant
3,"Charlotte, NC",35.227087,-80.843127,The Punch Room,35.226354,-80.841747,Hotel Bar
4,"Charlotte, NC",35.227087,-80.843127,Vapiano,35.226234,-80.844124,Restaurant


### Analyze Selected City

In [254]:
# one hot encoding
venueNonRestaurant_onehot = pd.get_dummies(venueNonRestaurant[['Venue Category']], prefix="", prefix_sep="")
venueNonRestaurant_onehot.head()
venueNonRestaurant_onehot.head()

,Bakery,Basketball Stadium,Café,Coffee Shop,Concert Hall,Creperie,Farmers Market,Gym,Gym / Fitness Center,Hotel,Hotel Bar,Park,Performing Arts Venue,Restaurant,Science Museum,Shopping Mall,Steakhouse,Theater
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [255]:
# set number of clusters
kclusters = NumberOfClusters+1
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venueNonRestaurant_onehot)
labels = kmeans.labels_
print(labels)
#venueNonRestaurant_onehot.head()
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
venueNonRestaurant.insert(0, 'Cluster Labels', kmeans.labels_)

venueNonRestaurant.head()

[0 0 1 0 1 0 0 0 1 2 1 0 0 1 0 0 2 1 0 0 1 1 1 0 0 0 1 0 0 2]


,Cluster Labels,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,"Charlotte, NC",35.227087,-80.843127,Blumenthal Performing Arts Center,35.227930,-80.841951,Performing Arts Venue
1,0,"Charlotte, NC",35.227087,-80.843127,Belk Theater,35.227711,-80.841663,Concert Hall
2,1,"Charlotte, NC",35.227087,-80.843127,The Capital Grille,35.228216,-80.841974,Restaurant
3,0,"Charlotte, NC",35.227087,-80.843127,The Punch Room,35.226354,-80.841747,Hotel Bar
4,1,"Charlotte, NC",35.227087,-80.843127,Vapiano,35.226234,-80.844124,Restaurant


### Create Map

In [256]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster,categories in zip(venueNonRestaurant['Venue Latitude'], venueNonRestaurant['Venue Longitude'], venueNonRestaurant['Venue'], venueNonRestaurant['Cluster Labels'],venueNonRestaurant['Venue Category']):
    labelName = "Venue Name: " + str(poi) + ' Category: ' + str(categories)
    label = folium.Popup(labelName, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [131]:
venueNonRestaurant.loc[venueNonRestaurant['Cluster Labels'] == 0, venueNonRestaurant.columns[[1] + list(range(5, venueNonRestaurant.shape[1]))]]

,Neighborhood,Venue Latitude,Venue Longitude,Venue Category
0,"Durham, NC",35.997861,-78.902637,Indie Movie Theater
1,"Durham, NC",35.996575,-78.901907,Ice Cream Shop
2,"Durham, NC",35.996998,-78.903716,Pizza Place
3,"Durham, NC",35.994690,-78.900868,Cocktail Bar
4,"Durham, NC",35.996927,-78.903894,Sandwich Place
5,"Durham, NC",35.993701,-78.902190,Concert Hall
6,"Durham, NC",36.000480,-78.901581,Farmers Market
7,"Durham, NC",35.991532,-78.904721,Baseball Stadium
9,"Durham, NC",35.996966,-78.900542,Cocktail Bar
12,"Durham, NC",35.997015,-78.904597,Deli / Bodega


In [132]:
venueNonRestaurant.loc[venueNonRestaurant['Cluster Labels'] == 1, venueNonRestaurant.columns[[1] + list(range(5, venueNonRestaurant.shape[1]))]]

,Neighborhood,Venue Latitude,Venue Longitude,Venue Category
10,"Durham, NC",35.995968,-78.901803,Hotel
23,"Durham, NC",35.997036,-78.900918,Hotel
25,"Durham, NC",35.996397,-78.901221,Hotel


In [133]:
venueNonRestaurant.loc[venueNonRestaurant['Cluster Labels'] == 2, venueNonRestaurant.columns[[1] + list(range(5, venueNonRestaurant.shape[1]))]]

,Neighborhood,Venue Latitude,Venue Longitude,Venue Category
8,"Durham, NC",35.997263,-78.901195,Restaurant
11,"Durham, NC",35.996965,-78.904849,Restaurant
19,"Durham, NC",35.995354,-78.900919,Restaurant
24,"Durham, NC",36.000656,-78.908853,Restaurant
